## Deliverable 2. Create a Customer Travel Destinations Map.

In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import sys

sys.path.append('../../API_Keys')

# Import API key
from config import gmaps_key

# Configure gmaps API key
gmaps.configure(api_key=gmaps_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Fuerte Olimpo,PY,-21.0415,-57.8738,92.03,25,7,12.80,clear sky
1,1,Barrow,US,71.2906,-156.7887,39.22,87,40,9.22,smoke
2,2,Rikitea,PF,-23.1203,-134.9692,74.05,71,2,4.18,clear sky
3,3,Kodiak,US,57.7900,-152.4072,54.91,94,100,3.44,light rain
4,4,Verkhoyansk,RU,67.5447,133.3850,72.90,71,100,12.59,overcast clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))

max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & 
                                       (city_data_df["Max Temp"] <= max_temp)]

preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
13,13,Ugoofaaru,MV,5.6667,73.0000,82.26,79,86,23.96,light rain
15,15,Celestun,MX,20.8667,-90.4000,84.34,63,53,4.32,broken clouds
16,16,Hithadhoo,MV,-0.6000,73.0833,82.38,73,72,3.60,broken clouds
19,19,Formosa Do Rio Preto,BR,-11.0483,-45.1931,84.18,30,3,9.75,clear sky
24,24,Sayat,TM,38.7839,63.8803,84.07,16,0,17.87,clear sky
25,25,Kavieng,PG,-2.5744,150.7967,82.36,81,100,15.64,light rain
27,27,Ormara,PK,25.2088,64.6357,85.62,84,89,6.55,light rain
30,30,Nikolskoye,RU,59.7035,30.7861,82.87,38,4,6.96,clear sky
31,31,Kapaa,US,22.0752,-159.3190,76.98,91,100,19.57,light rain
33,33,Hasaki,JP,35.7333,140.8333,75.60,84,100,9.35,overcast clouds


In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                282
City                   282
Country                281
Lat                    282
Lng                    282
Max Temp               282
Humidity               282
Cloudiness             282
Wind Speed             282
Current Description    282
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()

preferred_cities_df.count()

City_ID                281
City                   281
Country                281
Lat                    281
Lng                    281
Max Temp               281
Humidity               281
Cloudiness             281
Wind Speed             281
Current Description    281
dtype: int64

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
13,Ugoofaaru,MV,82.26,light rain,5.6667,73.0000,
15,Celestun,MX,84.34,broken clouds,20.8667,-90.4000,
16,Hithadhoo,MV,82.38,broken clouds,-0.6000,73.0833,
19,Formosa Do Rio Preto,BR,84.18,clear sky,-11.0483,-45.1931,
24,Sayat,TM,84.07,clear sky,38.7839,63.8803,
25,Kavieng,PG,82.36,light rain,-2.5744,150.7967,
27,Ormara,PK,85.62,light rain,25.2088,64.6357,
30,Nikolskoye,RU,82.87,clear sky,59.7035,30.7861,
31,Kapaa,US,76.98,light rain,22.0752,-159.3190,
33,Hasaki,JP,75.60,overcast clouds,35.7333,140.8333,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": gmaps_key
}

# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    
    lat = row["Lat"]
    
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
   
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
    try:
        
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        
        print("Hotel not found. Skipping...")

Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...


In [11]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
13,Ugoofaaru,MV,82.26,light rain,5.6667,73.0000,Ungoofaaru Inn
15,Celestun,MX,84.34,broken clouds,20.8667,-90.4000,Posada Lilia
16,Hithadhoo,MV,82.38,broken clouds,-0.6000,73.0833,Scoop Guest House
19,Formosa Do Rio Preto,BR,84.18,clear sky,-11.0483,-45.1931,Porto Center Hotel
24,Sayat,TM,84.07,clear sky,38.7839,63.8803,
25,Kavieng,PG,82.36,light rain,-2.5744,150.7967,Nusa Island Retreat
27,Ormara,PK,85.62,light rain,25.2088,64.6357,Kenara Colony House for Rent
30,Nikolskoye,RU,82.87,clear sky,59.7035,30.7861,Tourist House - Sablino
31,Kapaa,US,76.98,light rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
33,Hasaki,JP,75.60,overcast clouds,35.7333,140.8333,Inubosaki Kanko Hotel


In [12]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"] != '')]

clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
13,Ugoofaaru,MV,82.26,light rain,5.6667,73.0000,Ungoofaaru Inn
15,Celestun,MX,84.34,broken clouds,20.8667,-90.4000,Posada Lilia
16,Hithadhoo,MV,82.38,broken clouds,-0.6000,73.0833,Scoop Guest House
19,Formosa Do Rio Preto,BR,84.18,clear sky,-11.0483,-45.1931,Porto Center Hotel
25,Kavieng,PG,82.36,light rain,-2.5744,150.7967,Nusa Island Retreat
27,Ormara,PK,85.62,light rain,25.2088,64.6357,Kenara Colony House for Rent
30,Nikolskoye,RU,82.87,clear sky,59.7035,30.7861,Tourist House - Sablino
31,Kapaa,US,76.98,light rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
33,Hasaki,JP,75.60,overcast clouds,35.7333,140.8333,Inubosaki Kanko Hotel
35,Buala,SB,80.42,light rain,-8.1450,159.5921,Maringe Lagoon Lodge


In [13]:
# 8a. Create the output File (CSV)

output_data_file = ('WeatherPy_vacation.csv')

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>

<dt>Hotel Name</dt><dd>{Hotel Name}</dd>

<dt>City</dt><dd>{City}</dd>

<dt>Country</dt><dd>{Country}</dd>

<dt>Current Weather</dt><dd>{Current Description}</dd>

</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# 11b. Display the figure

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))